In [1]:
import torch
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import classification_report, confusion_matrix

In [2]:
transform_train = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),        # Randomly flip images horizontally for augmentation
    transforms.RandomRotation(15),
    transforms.Normalize(mean = [0.5]*3, std = [0.5]*3)
])

transform_test = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.5]*3, std = [0.5]*3)
])

In [3]:
train_dataset = datasets.ImageFolder(root = './train/',transform = transform_train)
test_dataset = datasets.ImageFolder(root = './test/',transform = transform_test)

train_size = int(0.7 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

In [4]:
def batch(batchsize):
    
    train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batchsize, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batchsize, shuffle=False)

    return train_loader, val_loader, test_loader

def create_cnn_model(filter_config):
    class CustomCNN(nn.Module):
        def __init__(self):
            super(CustomCNN, self).__init__()
            f1, f2, f3 = filter_config
            self.f3 = f3 
            self.conv1 = nn.Conv2d(3, f1, 3, padding=1)
            self.conv2 = nn.Conv2d(f1, f2, 3, padding=1)
            self.conv3 = nn.Conv2d(f2, f3, 3, padding=1)
            self.pool = nn.MaxPool2d(2, 2)
            self.dropout = nn.Dropout(0.3)
            self.fc1 = nn.Linear(f3 * 12 * 12, 128)
            self.fc2 = nn.Linear(128, 4)

        def forward(self, x):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = self.pool(F.relu(self.conv3(x)))
            x = x.view(-1, self.f3 * 12 * 12)
            x = self.dropout(F.relu(self.fc1(x)))
            x = self.fc2(x)
            return x

    return CustomCNN()

def train_model(model, criterion, optimizer, train_loader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        acc = 100. * correct / total
        print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Accuracy: {acc:.2f}%")

def full_evaluation(model, loader, dataset_name):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print(f"\n {dataset_name} Evaluation")
    print("-" * 40)
    print("Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))
    print("Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

In [5]:
configs = [
    ([32, 64, 128], "Run 1"),
    ([32, 64, 64],  "Run 2"),
    ([16, 32, 64],  "Run 3"),
    ([64, 64, 64],  "Run 4"),
    ([128, 128, 128], "Run 5"),
    ([32, 32, 32],  "Run 6"),
]

batch_sizes = [16, 32, 64]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for filters, label in configs:
    for bsz in batch_sizes:
        train_loader, val_loader, test_loader = batch(bsz)
        
        print(f"\n{label} {filters} (Batch size: {bsz})\n{'-' * 30}")
        model = create_cnn_model(filters).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        train_model(model, criterion, optimizer, train_loader)
        full_evaluation(model, val_loader, "Validation Set")
        full_evaluation(model, test_loader, "Test Set")
        
        print('-' * 50)


Run 1 [32, 64, 128] (Batch size: 16)
------------------------------
Epoch 1, Loss: 57.0301, Accuracy: 47.50%
Epoch 2, Loss: 37.8145, Accuracy: 71.25%
Epoch 3, Loss: 30.4358, Accuracy: 78.62%
Epoch 4, Loss: 26.2018, Accuracy: 81.62%
Epoch 5, Loss: 23.2500, Accuracy: 82.75%
Epoch 6, Loss: 20.9434, Accuracy: 85.62%
Epoch 7, Loss: 20.3382, Accuracy: 85.12%
Epoch 8, Loss: 19.6672, Accuracy: 86.88%
Epoch 9, Loss: 17.3744, Accuracy: 88.12%
Epoch 10, Loss: 16.4521, Accuracy: 89.62%

 Validation Set Evaluation
----------------------------------------
Classification Report:
              precision    recall  f1-score   support

       apple       0.95      0.93      0.94        61
      banana       0.84      0.92      0.88        52
       mixed       0.96      0.88      0.92        58
      orange       0.93      0.94      0.94        69

    accuracy                           0.92       240
   macro avg       0.92      0.92      0.92       240
weighted avg       0.92      0.92      0.92     